In [71]:
##Liberias

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import glob
import os

t_path = "C:/Users/diego/Downloads/Ciclo 2021-II/TFII/juez-bot-main/1.1.Resoluciones"

t_list = []
for year in os.listdir(t_path):
    
    c_path = t_path + "/" + year
    
    year = year[-4:]
    n_resol = len(os.listdir(c_path))
    t_list.append((year, n_resol))

for x in sorted(t_list, key=lambda x: x[0]):
    print(x)

In [95]:
df_cargos_desc = pd.read_excel("cargos_categorias.xlsx", 
                               sheet_name=0, 
                               usecols=[0,1,2,3])
# crear lista de los disintos cargos
cargos_list = list(df_cargos_desc["CÓDIGO"])

#df_cargos_desc.head()

In [96]:
df_cargos_multicat = pd.read_excel("res_hechos.xlsx", sheet_name=0, 
                                   usecols=[0, 1, 2, 3])

df_cargos_multicat["id_resol"] = \
    df_cargos_multicat["id_resol"]. \
        str.replace("–", "-"). \
        str.replace(" ", ""). \
        str.replace(".", "-"). \
        str.replace("SC2", "SPC"). \
        str.replace("TDC", "SPC")

df_cargos_multicat.info()
#df_cargos_multicat.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id_resol  288 non-null    object
 1   año       288 non-null    int64 
 2   cargos    288 non-null    object
 3   códigos   287 non-null    object
dtypes: int64(1), object(3)
memory usage: 9.1+ KB


In [97]:
#extrayendo data que proviene de 1. Cleaning-Resoluciones-Completas
df_resoluciones = pd.read_csv("res_multas.csv")

df_resoluciones["id_resol"] = \
    df_resoluciones["id_resol"]. \
        str.replace("–", "-"). \
        str.replace(" ", ""). \
        str.replace(".", "-"). \
        str.replace("SC2", "SPC"). \
        str.replace("TDC", "SPC")

df_resoluciones = df_resoluciones[["Año", "id_resol", "Empresa denunciada", "Multas", "Medidas correctivas"]]
df_resoluciones.info()
#df_resoluciones.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 301 entries, 0 to 300
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Año                  301 non-null    int64 
 1   id_resol             301 non-null    object
 2   Empresa denunciada   301 non-null    object
 3   Multas               301 non-null    object
 4   Medidas correctivas  301 non-null    object
dtypes: int64(1), object(4)
memory usage: 11.9+ KB


In [98]:
df_resol_comp = df_cargos_multicat.merge(right=df_resoluciones, how="inner", on="id_resol")
df_resol_comp = df_resol_comp.drop(["Año"], axis=1)

df_resol_comp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281 entries, 0 to 280
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_resol             281 non-null    object
 1   año                  281 non-null    int64 
 2   cargos               281 non-null    object
 3   códigos              280 non-null    object
 4   Empresa denunciada   281 non-null    object
 5   Multas               281 non-null    object
 6   Medidas correctivas  281 non-null    object
dtypes: int64(1), object(6)
memory usage: 17.6+ KB


In [99]:
filtro = (df_resol_comp["Multas"] == "0") & (df_resol_comp["Medidas correctivas"] == "0")

df_resol_comp["dec_fav_denunciante"] = (~filtro).astype("int8")
df_resol_comp.sample(5)

,id_resol,año,cargos,códigos,Empresa denunciada,Multas,Medidas correctivas,dec_fav_denunciante
253,1466-2006/SPC-INDECOPI,2006,realizó un viaje desde la ciudad de Lima a la ...,ser2,Empresa de Transporte Flores Hermanos S.R.L.,"0,5 UIT Pago de costas y costos","US$429,13",1
249,0071-2006/SPC-INDECOPI,2006,se acercó a las oficinas de la denunciada\nen ...,"ser2, ser3",PAREDES ESTRELLA CARGO S.A.,"0,5 UIT","Pago de S/.1,650",1
112,0515-2011/SPC-INDECOPI,2011,Se denuncio lo siguiente\n(i) El 23 de febrero...,"ser2, ser7",TRANSPORTES CRUZ DEL SUR S.A.C.,0.5 UIT,5000,1
276,1133-2010/SPC-INDECOPI,2010,(i) El 3 de abril de 2009 contrató los servic...,"ser1, ser7, ser13",Empresa de Transportes Expreso Cial S.A.C.,2 UIT Pago de costas y costos,S/.550.00,1
134,1921-2011/SPC-INDECOPI,2011,"(i) El 3 de julio de 2009, adquirió un boleto ...","ser2, ser5, o5",MERLY ROXANA GUEVARA BARRETO,2 UIT,180,1


# Pre-procesamiento de las resoluciones

In [100]:
import re
from nltk.corpus import stopwords

In [101]:
def limpieza_texto(string):
    string = string.lower()
    string = string.replace("\n", " ")
    list_words = re.findall(r'[a-z0-9ñáéíóúü-]+', string)
    list_words = [x for x in list_words if x not in stopwords.words('spanish')]
    string = " ".join(list_words)
    return string

In [102]:
# re.findall(r'[a-z0-9ñáéíóúü-]+', "la resolución 123-a, con código asdf4.")
limpieza_texto("la resolución 123-a,\ncon código asdf4.")

'resolución 123-a código asdf4'

In [103]:
df_resol_comp["cargos limpio"] = \
    df_resol_comp["cargos"].apply(limpieza_texto)

In [104]:
df_resol_comp["cargos limpio"]

0      luego contratar servicios denunciada 26 octubr...
1      cláusula 11 incluida boletos viaje establecía ...
2      cláusula 11 incluida boletos viaje establecía ...
3      empresa implementó establecimiento comercial l...
4      empresa brindado servicio idóneo ruta trujillo...
                             ...                        
276    i 3 abril 2009 contrató servicios expreso cial...
277    señaló denunciado brindado servicio transporte...
278    señaló 29 abril 2009 contrató servicios denunc...
279    denuncio viaje arequipa cusco realizado 22 jun...
280    viaje lima piura efectuado 27 febrero 2005 equ...
Name: cargos limpio, Length: 281, dtype: object

In [105]:
df_resol_comp["códigos"] = \
    df_resol_comp["códigos"].apply(lambda x: x.replace(" ", "") if isinstance(x, str) else x )

In [106]:
df_resol_comp.to_csv("res_completo.csv", encoding="utf-8")